In [1]:
import azureml.core

print("SDK Version:", azureml.core.VERSION)
subscription_id = "*add Azure subscription ID here*"
resource_group = "containers"
workspace_name = "mlworkspace"
workspace_region = "eastus2"

SDK Version: 1.0.10

In [2]:
# Import the Workspace class and check the Azure ML SDK version.
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      exist_ok=True)
ws.get_details()

Falling back to use azure cli credentials. This fall back to use azure cli credentials will be removed in the next release. 
Make sure your code doesn't require 'az login' to have happened before using azureml-sdk, except the case when you are specifying AzureCliAuthentication in azureml-sdk.
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code D2W37WQH4 to authenticate.
Interactive authentication successfully completed.
 Out[ 2 ]: 
{'identityType': 'SystemAssigned',
 'friendlyName': 'mlworkspace',
 'location': 'eastus2',
 'containerRegistry': '/subscriptions/c8a23972-1b42-43fa-9bda-92e665014f30/resourcegroups/containers/providers/microsoft.containerregistry/registries/mlworkspacrjjrhpxid',
 'keyVault': '/subscriptions/c8a23972-1b42-43fa-9bda-92e665014f30/resourcegroups/containers/providers/microsoft.keyvault/vaults/mlworkspkeyvaultmgzlbvxo',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'description': '',
 'workspaceid': '08e78cca-c25c-4bc2-b0b0-b265ca6eedc4',
 'applicationInsights': '/subscriptions/c8a23972-1b42-43fa-9bda-92e665014f30/resourcegroups/containers/providers/microsoft.insights/components/mlworkspinsightsezrfjlfo',
 'name': 'mlworkspace',
 'id': '/subscriptions/c8a23972-1b42-43fa-9bda-92e665014f30/resourceGroups/containers/providers/Microsoft.MachineLearningServices/workspaces/mlworkspace',
 'identityTenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'identityPrincipalId': '7a1e4299-c936-49b2-be8a-5f017c322740',
 'storageAccount': '/subscriptions/c8a23972-1b42-43fa-9bda-92e665014f30/resourcegroups/containers/providers/microsoft.storage/storageaccounts/mlworkspstorageoryypivl',
 'creationTime': '2019-02-05T13:57:16.8390709+00:00'}

In [3]:
#Configuring Your Local Environment
#You can validate that you have access to the specified workspace and write a configuration file to the default configuration location, ./aml_config/config.json.

from azureml.core import Workspace

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

# Persist the subscription id, resource group name, and workspace name in aml_config/config.json.
ws.write_config()

Wrote the config file config.json to: /databricks/driver/aml_config/config.json

In [4]:
#create folder to host sample projects
import os

sample_projects_folder = './sample_projects'

if not os.path.isdir(sample_projects_folder):
    os.mkdir(sample_projects_folder)
    
print('Sample projects will be created in {}.'.format(sample_projects_folder))

Sample projects will be created in ./sample_projects.

In [5]:
import pickle
import sys
import os
import pandas
import numpy as np
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve

from azureml.core import Run
# from utils import load_data

# create the outputs folder
os.makedirs('./outputs', exist_ok=True)

url = "https://raw.githubusercontent.com/TheovanKraay/mldata/master/iris.csv"
names = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width', 'Species']
iris = pandas.read_csv(url)

# load features and labels
X, Y = iris[['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']].values, iris['Species'].values

# split data 65%-35% into training set and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.35, random_state=0)

# change regularization rate and you will likely get a different accuracy
reg = 1

# load regularization rate from argument if present
if len(sys.argv) > 1:
   reg = float(sys.argv[1])
    
# get hold of the current run
run = Run.get_context()

print("Regularization rate is {}".format(reg))

# train a logistic regression model on the training set
clf1 = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=5).fit(X_train, Y_train)
print (clf1)

# evaluate the test set
accuracy = clf1.score(X_test, Y_test)
print ("Accuracy is {}".format(accuracy))

# serialize the model on disk in the special 'outputs' folder
print ("Export the model to model.pkl")
f = open('model.pkl', 'wb')
pickle.dump(clf1, f)
f.close()

Regularization rate is 33375.0
LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
 intercept_scaling=1, max_iter=100, multi_class='multinomial',
 n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
 tol=0.0001, verbose=0, warm_start=False)
Accuracy is 0.9811320754716981
Export the model to model.pkl

In [6]:
from azureml.core.model import Model

model = Model.register(model_path = "model.pkl",
                       model_name = "simpleml",
                       tags = {"key": "0.1"},
                       description = "test",
                       workspace = ws)

Registering model simpleml

In [7]:
%%writefile score.py
import pickle
import json
import numpy
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model


def init():
    global model
    model_path = Model.get_model_path(model_name = 'simpleml')
    
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    #model = clf1

def run(rawdata):
    try:
        data = json.loads(rawdata)['data']
        data = numpy.array(data)
        result = model.predict(data)
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    return json.dumps({"result":result.tolist()})

Writing score.py

In [8]:
#Create a YAML File for the Environment

from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn'], pip_packages=['azureml-sdk[automl]'])

conda_env_file_name = 'mydeployenv.yml'
myenv.save_to_file('.', conda_env_file_name)

Out[ 8 ]: 'mydeployenv.yml'

In [9]:
#Create ACI config
#deploy to ACI
from azureml.core.webservice import AciWebservice, Webservice

myaci_config = AciWebservice.deploy_configuration(
    cpu_cores = 2, 
    memory_gb = 2, 
    tags = {'name':'Databricks Azure ML ACI'}, 
    description = 'This is for simple regression model')

In [10]:
#Deploy the Image as a Web Service on Azure Container Instance

# this will take 10-15 minutes to finish

service_name = "tvkmlwebservice"
runtime = "spark-py" 
driver_file = "score.py"
my_conda_file = "mydeployenv.yml"

# image creation
from azureml.core.image import ContainerImage
myimage_config = ContainerImage.image_configuration(execution_script = driver_file, 
                                    runtime = runtime, 
                                    conda_file = 'mydeployenv.yml')

# Webservice creation
myservice = Webservice.deploy_from_model(
  workspace=ws, 
  name=service_name,
  deployment_config = myaci_config,
  models = [model],
  image_config = myimage_config
    )

myservice.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image tvkmlwebservice:1, operation "Succeeded"
Creating service
Running....................
SucceededACI service creation operation finished, operation "Succeeded"

In [11]:
#for using the Web HTTP API 
print(myservice.scoring_uri)

http://52.151.224.35:80/score